<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
From b1f539eb00d42bfd494b011fdaf2bbe760ca6683 Mon Sep 17 00:00:00 2001
From: zhengjun10 <zhengjun10@huawei.com>
Date: Thu, 24 Jun 2021 14:14:21 +0800
Subject: [PATCH] fix converter bug under train and add demo time memory print

---
 mindspore/lite/examples/transfer_learning/src/net_runner.cc | 10 ++++++++++
 mindspore/lite/tools/anf_exporter/anf_exporter.cc           | 11 ++++++++++-
 2 files changed, 20 insertions(+), 1 deletion(-)

diff --git a/mindspore/lite/examples/transfer_learning/src/net_runner.cc b/mindspore/lite/examples/transfer_learning/src/net_runner.cc
index d400d95..3e5e87b 100644
--- a/mindspore/lite/examples/transfer_learning/src/net_runner.cc
+++ b/mindspore/lite/examples/transfer_learning/src/net_runner.cc
@@ -16,9 +16,11 @@
 
 #include "src/net_runner.h"
 #include <getopt.h>
+#include <malloc.h>
 #include <algorithm>
 #include <cmath>
 #include <cstring>
+#include <chrono>
 #include <fstream>
 #include <iostream>
 #include "include/context.h"
@@ -183,6 +185,7 @@ int NetRunner::TrainLoop() {
   session_->Train();
   float min_loss = 1000.;
   float max_acc = 0.;
+  auto start_time = std::chrono::high_resolution_clock::now();
   for (int i = 0; i < cycles_; i++) {
     FillInputData(ds_.train_data());
     session_->RunGraph(nullptr, verbose_ ? after_callback : nullptr);
@@ -205,6 +208,13 @@ int NetRunner::TrainLoop() {
       if (acc > kThreshold) return 0;
     }
   }
+  auto end_time = std::chrono::high_resolution_clock::now();
+  auto time_cost = std::chrono::duration<double, std::milli>(end_time - start_time);
+  if (cycles_ > 0) {
+    std::cout << "AvgRunTime: " << time_cost.count() / cycles_ << " ms" << std::endl;
+  }
+  struct mallinfo info = mallinfo();
+  std::cout << "Total allocation: " << info.arena + info.hblkhd << std::endl;
   return 0;
 }
 
diff --git a/mindspore/lite/tools/anf_exporter/anf_exporter.cc b/mindspore/lite/tools/anf_exporter/anf_exporter.cc
index 31e038c..33a1d7b 100644
--- a/mindspore/lite/tools/anf_exporter/anf_exporter.cc
+++ b/mindspore/lite/tools/anf_exporter/anf_exporter.cc
@@ -505,7 +505,16 @@ int AnfExporter::ConvertInputParameter(const CNodePtr &cnode, size_t index, cons
   MS_ASSERT(param_node != nullptr);
   auto key = std::make_pair(param_node, 0);
   if (node_id_map_.find(key) != node_id_map_.end()) {
-    op_node->inputIndex.emplace_back(node_id_map_[key]);
+    auto tensor_index = node_id_map_[key];
+    auto &tensor = meta_graphT->allTensors.at(tensor_index);
+    // train model weight tensor is sharing need hardcode KHWC
+    if ((opt::CheckPrimitiveType(cnode, prim::kPrimConv2DFusion) ||
+         opt::CheckPrimitiveType(cnode, opt::kPrimConv2DBackpropInputFusion) ||
+         opt::CheckPrimitiveType(cnode, prim::kPrimConv2dTransposeFusion)) &&
+        index == 2) {
+      tensor->format = static_cast<schema::Format>(mindspore::KHWC);
+    }
+    op_node->inputIndex.emplace_back(tensor_index);
     return RET_OK;
   }
   DataInfo data_info;
-- 
2.7.4

